Cómo Funciona
Modelo LLM:

Utilizamos un modelo de Hugging Face (distilbert-base-uncased) configurado para tareas de clasificación de texto.
Este modelo devuelve un puntaje de similitud entre el texto procesado (consulta + texto objetivo).
División en Palabras:

La consulta se divide en palabras para que SHAP pueda evaluar la importancia de cada una.
predict_text:

Genera texto filtrado basado en las palabras activas y calcula un puntaje de similitud con el modelo LLM.
Referencia SHAP:

Una matriz aleatoria de referencia ayuda a SHAP a generar explicaciones al comparar diferentes combinaciones de palabras.
Visualización:

Se utiliza shap.force_plot para mostrar cómo cada palabra contribuye al puntaje de similitud.

In [ ]:
%pip install shap transformers numpy

In [ ]:
import shap
import numpy as np
from transformers import pipeline

: 

In [ ]:
# 1. Cargar un modelo preentrenado de Transformers (pipeline de análisis de sentimientos)
sentiment_analyzer = pipeline("sentiment-analysis")

# 2. Crear un conjunto de datos de prueba
texts = [
    "I love using machine learning models!",
    "The weather is terrible today.",
    "This movie was okay, not the best, but not the worst.",
]

# 3. Definir una función que adapte los datos de entrada para SHAP
def model_predict(texts):
    predictions = sentiment_analyzer(texts)
    # Convertimos los resultados en probabilidades para SHAP
    return np.array([[pred['score'] if pred['label'] == 'POSITIVE' else 1 - pred['score']] for pred in predictions])

# 4. Crear un explicador SHAP con KernelExplainer
explainer = shap.KernelExplainer(model_predict, np.array(texts, dtype=object))

# 5. Generar explicaciones SHAP para un texto específico
shap_values = explainer.shap_values(["I am feeling great about this!"])

# 6. Visualizar los valores SHAP
print("Valores SHAP para el texto:")
shap.force_plot(
    explainer.expected_value[0],
    shap_values[0],
    feature_names=["I am feeling great about this!"],
)


: 